In [1]:
from google.colab import files

uploaded = files.upload()
# use .s2 datas

Saving reddit_matched_sample_test.parquet to reddit_matched_sample_test.parquet
Saving reddit_matched_sample_train.parquet to reddit_matched_sample_train.parquet


In [ ]:
#!pip install pyarrow

In [2]:
import pandas as pd
import pyarrow

#df_train = pd.read_parquet("reddit_data_sample_train_s2.parquet") # s on pienempi data, 15892 riviä
#df_eval = pd.read_parquet("reddit_data_sample_test_s2.parquet")

df_train = pd.read_parquet("reddit_matched_sample_train.parquet") # Pyritty saamaan lähelle GAD dataa, 46355 riviä
df_eval = pd.read_parquet("reddit_matched_sample_test.parquet")


In [3]:
df_train["labels"] = df_train["labels"].astype("int")
df_eval["labels"] = df_eval["labels"].astype("int")

df_train["text"] = df_train["text"].astype("str")
df_eval["text"] = df_eval["text"].astype("str")

df_train.head()

,labels,text
0,0,"oatlyn iKaffe on maitoa parempaa kahvissa, en ..."
1,1,Huomasitko että Nyssen sivuilla ei lue että ra...
2,0,Helpoin tapa on tehdä asp tili ja säästää sill...
3,0,No jos luetaan asunnot pois niin velkaa 280k n...
4,0,"Mutta jos säästät aspille enemmän, niin voit t..."


In [4]:
#from transformers import AutoTokenizer
from spacy.lang.fi import Finnish

#tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base') # Ei näin
tokenizer = Finnish().tokenizer

In [5]:
!pip install imodelsx

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.5/115.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
from imodelsx import AugLinearClassifier
import numpy as np

# fit model
m = AugLinearClassifier(
    tokenizer_ngrams = tokenizer,
    checkpoint='FacebookAI/xlm-roberta-base',
    ngrams=2, # Testaa tällä
)

m.fit(df_train['text'], df_train['labels'])

# Aikaa kului: 20 min 21 s (josta 11 min embeddingien laskemiseen, 7,5 min caching linear coefs)
# Toimimattomalla mallilla (väärä tokenizer): 4 min 18 s

# matched data
# Aikaa kului: 1 h 52 min (josta 1:10 min embeddingien laskemiseen, 42 min caching linear coefs)

initializing model...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

...

100%|██████████| 52245/52245 [1:10:13<00:00, 12.40it/s]


set up linear model...
caching linear coefs...


100%|██████████| 2667/2667 [42:36<00:00,  1.04it/s]


	After caching, len(coefs_dict_)=1365324, up from 0


AugLinearClassifier(checkpoint='FacebookAI/xlm-roberta-base',
                    tokenizer_ngrams=<spacy.tokenizer.Tokenizer object at 0x7cfe33c11bd0>)

In [ ]:
#m.cache_linear_coefs(df_eval['text'])

	Nothing to update!


In [7]:
# predict
preds = m.predict(df_eval['text'])
print('acc_val', np.mean(preds == df_eval['labels'])) # Huomaa, että tämä printtaa accuracyn -> pitäisi olla oma funktio, joka ennustaa todennäköisyydet

# Aikaa kului: 0s
# accuracy 0.69

# Matched mallilla 0.73

acc_val 0.7346938775510204


/usr/local/lib/python3.11/dist-packages/imodelsx/auglinear/auglinear.py:404: UserWarning: Saw an unseen ungram 125894 times. For better performance, call cache_linear_coefs on the test dataset before calling predict.
  warnings.warn(


In [8]:
# interpret
print('Total ngram coefficients: ', len(m.coefs_dict_))
print('Most positive ngrams')
for k, v in sorted(m.coefs_dict_.items(), key=lambda item: item[1], reverse=True)[:8]:
    print('\t', k, round(v, 2))
print('Most negative ngrams')
for k, v in sorted(m.coefs_dict_.items(), key=lambda item: item[1])[:8]:
    print('\t', k, round(v, 2))

Total ngram coefficients:  1365324
Most positive ngrams
	 turistiakin . 1.4
	 hämeenkatua . 1.31
	 rautatieasemaltaki keskustorille 1.3
	 puistokatu . 1.28
	 pyörätie Hämeenkadun 1.27
	 Tampereen Raitiotie 1.24
	 Rautatieasema-Valkea väliltä 1.22
	 keskusta . 1.22
Most negative ngrams
	 Sony WH-1000XM3:set -1.16
	 @ Nordea -1.15
	 valitse investment -1.14
	 superrahastoihin . -1.11
	 lainasumma . -1.07
	 Academic Nintendo -1.07
	 rahasto ostaa -1.06
	 sijoittajan kannalta -1.06


In [9]:
probs = m.predict_proba(df_eval['text'])

# 0 s

In [10]:
probs[0:9]

array([[0.99278328, 0.00721672],
       [0.68390077, 0.31609923],
       [0.07306374, 0.92693626],
       [0.26554621, 0.73445379],
       [0.36097322, 0.63902678],
       [0.7868457 , 0.2131543 ],
       [0.07544549, 0.92455451],
       [0.72212289, 0.27787711],
       [0.99135772, 0.00864228]])

In [11]:
# probabilities into dataframe
probs_df = pd.DataFrame(probs)

probs_df.head()

,0,1
0,0.992783,0.007217
1,0.683901,0.316099
2,0.073064,0.926936
3,0.265546,0.734454
4,0.360973,0.639027


In [ ]:
# Save probabilities

from google.colab import  drive

probs_df.to_csv('/content/drive/My Drive/Colab Notebooks/aug_imodel_probs.csv')

In [ ]:
# Save model

import pickle

path = "/content/drive/My Drive/Colab Notebooks/"

pickle_out = open(path + "aug_imodel", "wb") # huom + tässä!
pickle.dump(m, pickle_out)
pickle_out.close()

In [12]:
# !!!! Kokeile tallentaa malli ennen tätä steppiä !!!!





m.cache_linear_coefs(df_eval['text']) # 1 min 54 s
# ~4 min matched datalla

# Tämän stepin voisi kokeilla tehdä CPU:lla? Viekö paljon kauemman aikaa?
# (tarvii ehkä tallentaa malliobjekti tätä varten..?)

100%|██████████| 244/244 [03:55<00:00,  1.03it/s]


	After caching, len(coefs_dict_)=1490131, up from 1365324


In [13]:
preds = m.predict(df_eval['text'])
print('acc_val', np.mean(preds == df_eval['labels']))

# 0.77 -> parantui huomattavasti!
# 0.80 mathced datalla

acc_val 0.7987092272806559


In [14]:
probs = m.predict_proba(df_eval['text'])
probs[1:10]

array([[2.11916770e-02, 9.78808323e-01],
       [2.02860241e-06, 9.99997971e-01],
       [6.78227050e-11, 1.00000000e+00],
       [6.46224144e-04, 9.99353776e-01],
       [5.18939977e-01, 4.81060023e-01],
       [2.33432513e-04, 9.99766567e-01],
       [4.70802145e-01, 5.29197855e-01],
       [9.93291131e-01, 6.70886858e-03],
       [9.77314783e-01, 2.26852171e-02]])

In [ ]:
#m.save("/content/drive/My Drive/Colab Notebooks/")

coefs = m.coefs_dict_

In [ ]:
import pickle

path = "/content/drive/My Drive/Colab Notebooks/"

pickle_out = open(path + "aug_imodel", "wb") # huom + tässä!
pickle.dump(m, pickle_out)
pickle_out.close()

# SUCCES!!!

#pickle_out = open("aug_imodel", "wb") # Tämä vie tiedostonäkymään, mutta ei driveen
#pickle.dump(m, pickle_out)
#pickle_out.close()